# Imports

In [1]:
import csv  # for slang
import os
import re  # regex
import string  # punct
from pprint import pprint
!pip install emoji --upgrade
import emoji  # for emoji
import gensim
import keras
import lightgbm as lgb
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from gensim.models import Word2Vec
from IPython.display import Image
from matplotlib import pyplot as plt
from nltk.corpus import stopwords  # stopwords
from nltk.stem import PorterStemmer  # stemming
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn import svm, tree
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              GradientBoostingClassifier,
                              RandomForestClassifier, RandomForestRegressor,
                              StackingClassifier)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import (accuracy_score, auc, average_precision_score,
                             brier_score_loss, classification_report,
                             confusion_matrix, f1_score, fbeta_score,
                             make_scorer, plot_precision_recall_curve,
                             precision_recall_curve, precision_score,
                             recall_score, roc_auc_score, roc_curve)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     cross_val_score, train_test_split)
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.svm import SVC  # "Support vector classifier"
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb

# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from google.colab import files
uploaded = files.upload()
import nltk
nltk.download('stopwords')

%matplotlib inline

     |████████████████████████████████| 51kB 2.4MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=b32af3ac626fc6eefd4f161c3067400bebb1adf890ede9ef22903b14172b4929
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Saving slang.txt to slang.txt
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Homemade Classes and Functions

In [2]:
# Clean Text Class

class CleanText(BaseEstimator, TransformerMixin):
    
    def remove_mentions(self, input_text):
        '''
        Remove mentions, like @Mplamplampla
        '''
        return re.sub(r'@+', '', input_text)
    
    def remove_urls(self, input_text):
        '''
        Remove the urls mention in a tweet
        '''
        input_text  = ' '.join([w for w in input_text.split(' ') if '.com' not in w])
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        input_text = emoji.demojize(input_text)
        input_text = input_text.replace('_','')
        input_text = input_text.replace(':','')
        return input_text
    
    def possessive_pronouns(self, input_text):
        '''
        Remove the possesive pronouns, because otherwise after tokenization we will end up with a word and an s
        Example: government's --> ["government", "s"]
        '''
        return input_text.replace("'s", "")
    
    def characters(self, input_text):
        '''
        Remove special and redundant characters that may appear on a tweet and that don't really help in our analysis
        '''
        input_text = input_text.replace("\r", " ") # Carriage Return
        input_text = input_text.replace("\n", " ") # Newline
        input_text = " ".join(input_text.split()) # Double space
        input_text = input_text.replace('"', '') # Quotes
        return input_text
    
    def remove_punctuation(self, input_text):
        '''
        Remove punctuation and specifically these symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        '''
        punct = string.punctuation # string with all the punctuation symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)
    
    def remove_digits(self, input_text):
        '''
        Remove numbers
        '''
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        '''
        Convert all the sentences(words) to lowercase
        '''
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        '''
        Remove stopwords (refers to the most common words in a language)
        '''
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        '''
        Reduce the words to their stem
        '''
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
    def encode_decode(self, input_text):
        '''
        Remove weird characters that are result of encoding problems
        '''
        return  " ".join([k.encode("ascii", "ignore").decode() for k in input_text.split(" ")])
    
    
    def translator(self, input_text):
        '''
        Transform abbrevations to normal words
        Example: asap --> as soon as possible
        '''
        input_text = input_text.split(" ")
        j = 0
        for _str in input_text:
            # File path which consists of Abbreviations.
            fileName = "/content/slang.txt"
            # File Access mode [Read Mode]
            accessMode = "r"
            with open(fileName, accessMode) as myCSVfile:
                # Reading file as CSV with delimiter as "=", so that 
                # abbreviation are stored in row[0] and phrases in row[1]
                dataFromFile = csv.reader(myCSVfile, delimiter="=")
                # Removing Special Characters.
                _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
                for row in dataFromFile:
                    # Check if selected word matches short forms[LHS] in text file.
                    if _str.upper() == row[0]:
                        # If match found replace it with its appropriate phrase in text file.
                        input_text[j] = row[1]
                myCSVfile.close()
            j = j + 1
        
        return(' '.join(input_text))
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = (X.apply(self.translator)
                    .apply(self.remove_mentions)
                    .apply(self.remove_urls)
                    .apply(self.emoji_oneword)
                    .apply(self.possessive_pronouns)
                    .apply(self.remove_punctuation)
                    .apply(self.remove_digits)
                    .apply(self.encode_decode)
                    .apply(self.characters)
                    .apply(self.to_lower)
                    .apply(self.remove_stopwords)
                    .apply(self.stemming))
        return clean_X
    
    def transform_no_stem(self, X, **transform_params):
        clean_X = (X.apply(self.translator)
                    .apply(self.remove_mentions)
                    .apply(self.remove_urls)
                    .apply(self.emoji_oneword)
                    .apply(self.possessive_pronouns)
                    .apply(self.remove_punctuation)
                    .apply(self.remove_digits)
                    .apply(self.encode_decode)
                    .apply(self.characters)
                    .apply(self.to_lower)
                    .apply(self.remove_stopwords))
        return clean_X

# Read in Data and Create Train and Test Sets

In [3]:
tweets = pd.read_csv('https://github.com/anilkeshwani/StatLearnProj/raw/master/Iason/climate_change_tweets_sample-2020-05-16-17-57.csv')
tweets.head()

,username,user_handle,date,retweets,favorites,text,label
0,WWF Climate & Energy,climateWWF,2020-04-28,11,22,Economic recovery and national climate pledges...,0
1,WWF Climate & Energy,climateWWF,2020-04-22,6,16,"In this difficult time, it’s hard to connect w...",0
2,WWF Climate & Energy,climateWWF,2020-04-01,43,69,"The decision to postpone # COP26, is unavoidab...",0
3,WWF Climate & Energy,climateWWF,2020-03-30,24,30,Japan - the world’s fifth largest emitter of g...,0
4,WWF Climate & Energy,climateWWF,2020-03-30,22,40,How can countries include # NatureBasedSolutio...,0


In [4]:
ct = CleanText()
tweets["text"] = ct.fit_transform(tweets.text)
tweets = tweets.loc[(~tweets.text.isnull()), :]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(tweets.text, tweets.label, 
                                                    test_size=0.2, random_state=17, 
                                                    shuffle=True) # explicit default

[print(dat.head(3), dat.shape, end="\n"*2) for dat in [X_train, X_test, Y_train, Y_test]];

3641     insect integr part life environ increas heat c...
11837                                  believ climat chang
8450              climat model need worri programm set run
Name: text, dtype: object (14407,)

8375          climat scienc settl sustain stagnant
6110          call climat plan good start joebiden
16330    obama govern need save planet global warm
Name: text, dtype: object (3602,)

3641     0
11837    1
8450     1
Name: label, dtype: int64 (14407,)

8375     1
6110     0
16330    1
Name: label, dtype: int64 (3602,)



# Word Vectorisations

In [6]:
bigram_vectorizer = CountVectorizer(stop_words = 'english', 
                                    binary=True, 
                                    min_df = 10,
                                    ngram_range = (1,2)) # create bigrams
bigram_vectorizer.fit(X_train)

bigram_train = bigram_vectorizer.transform(X_train)
bigram_test = bigram_vectorizer.transform(X_test)

# Modelling

In [7]:
class Majority_vote():
    """
    averaging all models
    """
    def __init__(self,models,threshold=None):
        self.base_models = models
        if threshold:
            self.threshold = threshold
        else:
            self.threshold = 0.5
              
    
    def fit(self,X_train,y_train):
        for model in self.base_models:       
            model.fit(X_train,y_train)

    def predict_score(self,X):
        s = []
        for k,model in enumerate(self.base_models):
            pred = model.predict_proba(X)[:,1]
            s.append(pred)
        s = np.array(s)
        return(np.sum(s,axis=0)/len(self.base_models))
    
    def predict(self,X):
        score = self.predict_score(X)
        pred = np.zeros(X.shape[0])
        pred[score>=self.threshold] = 1
        return pred

In [9]:
! pip3 install catboost
import catboost as cb
estimators = [RandomForestClassifier(**{'bootstrap': True,'max_depth': None,'max_features': 'log2',
                                        'min_samples_leaf': 1,'min_samples_split': 10,'n_estimators': 300,
                                        'random_state':17}),
              SVC(**{'C': 100, 'gamma': 0.001, 'kernel': 'rbf','probability':True,'random_state':17}),
              LogisticRegression(**{'C':1,'max_iter':1000, 'multi_class':"ovr", 'solver':"saga",'random_state':17}),
              cb.CatBoostClassifier(**{"border_count": 200, "depth": 6, "l2_leaf_reg": 10,
                                       "learning_rate": 0.3, "n_estimators": 1000,'verbose':0,'random_state':17})]

     |████████████████████████████████| 64.8MB 67kB/s 


In [14]:
clf = Majority_vote(estimators)
clf.fit(bigram_train,Y_train)
#pred = clf.predict(XX_val)

In [22]:
uploaded = files.upload()

Saving Fridays4future.csv to Fridays4future.csv
Saving PrisonPlanet.csv to PrisonPlanet.csv
Saving StephenAtHome.csv to StephenAtHome.csv


In [23]:
Fridays4future = pd.read_csv("/content/Fridays4future.csv")
PrisonPlanet = pd.read_csv("/content/PrisonPlanet.csv")
StephenAtHome = pd.read_csv("/content/StephenAtHome.csv")

In [24]:
Fridays4future["text"] = ct.fit_transform(Fridays4future.text)
PrisonPlanet["text"] = ct.fit_transform(PrisonPlanet.text)
StephenAtHome["text"] = ct.fit_transform(StephenAtHome.text)

In [25]:
Fridays4future = bigram_vectorizer.transform(Fridays4future["text"])
PrisonPlanet = bigram_vectorizer.transform(PrisonPlanet["text"])
StephenAtHome = bigram_vectorizer.transform(StephenAtHome["text"])

# Predicting tweets outside our dataset 



> Fridays4future ( Believer )




![](https://drive.google.com/uc?export=view&id=1H5bsXG43D31g6W3RHlfywoZpX3ddlFG8)

In [27]:
clf.predict_score(Fridays4future)

array([0.11177787])

In [28]:
clf.predict(Fridays4future)

array([0.])



> PrisonPlanet ( Denier )

![](https://drive.google.com/uc?export=view&id=1DlMB5Xnfj8M9BwuacFGSO2zAj5eXNONX)


In [29]:
clf.predict_score(PrisonPlanet)

array([0.97389337])

In [30]:
clf.predict(PrisonPlanet)

array([1.])



> StephenAtHome 

This is intteresting because he probably is a believer however his tweet is ironic, so let's see what our classifier will find out 

![](https://drive.google.com/uc?export=view&id=1yKhnLqzAdYngpTmBbOx1mhx2l5s568N5)


In [31]:
clf.predict_score(StephenAtHome)

array([0.86837466])

In [32]:
clf.predict(StephenAtHome)

array([1.])